In [3]:
#  DFA/NFA Visual Simulator

# Install Dependencies
# !pip install graphviz pandas ipywidgets

# DFA Class Definition
class DFA:
    def __init__(self, states, alphabet, transitions, start_state, accept_states):
        self.states = states
        self.alphabet = alphabet
        self.transitions = transitions
        self.start_state = start_state
        self.accept_states = accept_states

    def run(self, input_string):
        current_state = self.start_state
        path = [current_state]
        for symbol in input_string:
            if symbol not in self.alphabet:
                raise ValueError(f"Invalid symbol '{symbol}'")
            current_state = self.transitions[current_state][symbol]
            path.append(current_state)
        return current_state in self.accept_states, path

    def run_step_by_step(self, input_string):
        current_state = self.start_state
        steps = [('', current_state)]
        for symbol in input_string:
            if symbol not in self.alphabet:
                raise ValueError(f"Invalid symbol '{symbol}'")
            current_state = self.transitions[current_state][symbol]
            steps.append((symbol, current_state))
        return current_state in self.accept_states, steps

# NFA Class Definition (with ε-transitions)
class NFA:
    def __init__(self, states, alphabet, transitions, start_state, accept_states):
        self.states = states
        self.alphabet = alphabet
        self.transitions = transitions
        self.start_state = start_state
        self.accept_states = accept_states

    def _epsilon_closure(self, states):
        stack = list(states)
        closure = set(states)
        while stack:
            state = stack.pop()
            for next_state in self.transitions.get(state, {}).get('', []):
                if next_state not in closure:
                    closure.add(next_state)
                    stack.append(next_state)
        return closure

    def run_step_by_step(self, input_string):
        steps = [('', list(self._epsilon_closure({self.start_state})))]
        current_states = self._epsilon_closure({self.start_state})
        for symbol in input_string:
            next_states = set()
            for state in current_states:
                for target in self.transitions.get(state, {}).get(symbol, []):
                    next_states.update(self._epsilon_closure({target}))
            current_states = next_states
            steps.append((symbol, list(current_states)))
        accepted = any(state in self.accept_states for state in current_states)
        return accepted, steps

    def run(self, input_string):
        current_states = self._epsilon_closure({self.start_state})
        current = [(state, [self.start_state, 'ε', state] if state != self.start_state else [state]) for state in current_states]

        for symbol in input_string:
            next_set = []
            for state, path in current:
                for next_state in self.transitions.get(state, {}).get(symbol, []):
                    closure = self._epsilon_closure({next_state})
                    for cs in closure:
                        next_path = path + [symbol, cs]
                        next_set.append((cs, next_path))
            current = next_set

        accepting_paths = [path for state, path in current if state in self.accept_states]
        all_paths = [path for _, path in current]
        return bool(accepting_paths), accepting_paths if accepting_paths else all_paths
import itertools

# Step 4: Convert NFA to DFA (Subset Construction)
from collections import deque

def nfa_to_dfa(nfa):
    dfa_states = {}
    dfa_transitions = {}
    start_set = frozenset(nfa._epsilon_closure({nfa.start_state}))
    queue = deque([start_set])
    dfa_state_names = {start_set: 'D0'}
    accept_states = set()
    state_count = 1

    while queue:
        current = queue.popleft()
        current_name = dfa_state_names[current]
        dfa_transitions[current_name] = {}

        if current & nfa.accept_states:
            accept_states.add(current_name)

        for symbol in nfa.alphabet:
            if symbol == '':
                continue
            next_set = set()
            for state in current:
                next_states = nfa.transitions.get(state, {}).get(symbol, [])
                for next_state in next_states:
                    next_set.update(nfa._epsilon_closure({next_state}))
            next_fset = frozenset(next_set)
            if next_fset not in dfa_state_names:
                dfa_state_names[next_fset] = f"D{state_count}"
                state_count += 1
                queue.append(next_fset)
            dfa_transitions[current_name][symbol] = dfa_state_names[next_fset]

    return DFA(
        states=set(dfa_transitions.keys()),
        alphabet=nfa.alphabet - {''},
        transitions=dfa_transitions,
        start_state='D0',
        accept_states=accept_states
    )

# Load DFA/NFA from JSON
import json

def load_automaton_from_json(json_str, automaton_type='DFA'):
    data = json.loads(json_str)
    states = set(data['states'])
    alphabet = set(data['alphabet'])
    transitions = data['transitions']
    start_state = data['start_state']
    accept_states = set(data['accept_states'])

    if automaton_type == 'DFA':
        return DFA(states, alphabet, transitions, start_state, accept_states)
    elif automaton_type == 'NFA':
        return NFA(states, alphabet, transitions, start_state, accept_states)
    else:
        raise ValueError("Unknown automaton type")

#  Graphviz Visualizer for DFA/NFA
from graphviz import Digraph
from IPython.display import Image, display, HTML

def visualize_automaton(automaton, title="Automaton", is_nfa=False):
    dot = Digraph()
    dot.attr(rankdir='LR', label=title, fontsize='12', labelloc='t')

    for state in automaton.states:
        shape = 'doublecircle' if state in automaton.accept_states else 'circle'
        dot.node(state, shape=shape)

    dot.node('', shape='none')
    dot.edge('', automaton.start_state)

    for src, transitions in automaton.transitions.items():
        for symbol, dst in transitions.items():
            if is_nfa and isinstance(dst, list):
                for d in dst:
                    label = symbol if symbol != '' else 'ε'
                    dot.edge(src, d, label=label)
            else:
                dot.edge(src, dst, label=symbol)

    filename = 'nfa_graph' if is_nfa else 'dfa_graph'
    dot.render(filename, format='png', cleanup=True)
    display(Image(filename=f'{filename}.png'))

#  DFA & NFA Animation
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os

def animate_path(path, automaton, filename_prefix="step", is_nfa=False):
    image_paths = []

    for i in range(len(path)):
        dot = Digraph()
        dot.attr(rankdir='LR')

        current_state = path[i] if isinstance(path[i], str) else path[i][-1]
        for s in automaton.states:
            if s == current_state:
                dot.node(s, shape='doublecircle' if s in automaton.accept_states else 'circle', style='filled', fillcolor='red')
            else:
                dot.node(s, shape='doublecircle' if s in automaton.accept_states else 'circle')

        dot.node('', shape='none')
        dot.edge('', automaton.start_state)

        for src, transitions in automaton.transitions.items():
            for sym, dst in transitions.items():
                if is_nfa and isinstance(dst, list):
                    for d in dst:
                        label = sym if sym != '' else 'ε'
                        dot.edge(src, d, label=label)
                else:
                    dot.edge(src, dst, label=sym)

        step_file = f"{filename_prefix}_{i}.png"
        dot.render(step_file, format='png', cleanup=True)
        image_paths.append(f"{step_file}.png")

    fig = plt.figure(figsize=(2,2))
    imgs = [plt.imread(path) for path in image_paths]

    def update(i):
        plt.clf()
        plt.imshow(imgs[i])
        plt.axis('off')
        plt.title(f"Step {i}")

    anim = animation.FuncAnimation(fig, update, frames=len(imgs), interval=1000, repeat=False)
    html_anim = HTML(anim.to_jshtml())

    for path in image_paths:
        os.remove(path)

    display(html_anim)
    plt.close(fig)
    return html_anim

from ipywidgets import Dropdown, Textarea, Text, Checkbox, VBox, Output, HBox, interactive_output
from IPython.display import display, Markdown, clear_output

# Example JSONs
example_jsons = {
    "DFA": '''{
  "states": ["q0", "q1"],
  "alphabet": ["0", "1"],
  "transitions": {
    "q0": {"0": "q0", "1": "q1"},
    "q1": {"0": "q1", "1": "q0"}
  },
  "start_state": "q0",
  "accept_states": ["q0"]
}''',
    "NFA": '''{
  "states": ["q0", "q1", "q2"],
  "alphabet": ["0", "1", ""],
  "transitions": {
    "q0": {"0": ["q0"], "": ["q1"]},
    "q1": {"1": ["q2"]},
    "q2": {}
  },
  "start_state": "q0",
  "accept_states": ["q2"]
}'''
}

# Widgets
automaton_type = Dropdown(options=["DFA", "NFA"], description="Type:")
input_string = Text(value="", description="Input:")
visualize = Checkbox(value=True, description="Visualize")
show_dfa = Checkbox(value=True, description="Simulate Equivalent DFA")
json_input = Textarea(value=example_jsons["DFA"], layout={"width": "800px", "height": "200px"}, description="")

# Output areas
trace_output = Output()
graph_output = Output()

# Automatically update JSON when automaton type changes
def on_type_change(change):
    json_input.value = example_jsons[change['new']]

automaton_type.observe(on_type_change, names='value')
def simulate(automaton_type, input_string, visualize, show_dfa):
    clear_output(wait=True)
    trace_output.clear_output()
    graph_output.clear_output()

    display(Markdown("### 📟 Paste or Edit the Automaton JSON Below:"))
    display(json_input)

    try:
        automaton = load_automaton_from_json(json_input.value, automaton_type)

        if automaton_type == "DFA":
          # --- DFA Simulation ---
          result, steps = automaton.run_step_by_step(input_string)
          display(Markdown(f"### ✅ DFA Result: {'**Accepted ✅**' if result else '**Rejected ❌**'}"))
          display(Markdown("---"))
          display(Markdown("#### 🔍 DFA Step-by-Step Trace"))

          with trace_output:
              for symbol, state in steps:
                  label = "Start" if symbol == '' else f"`{symbol}`"
                  display(Markdown(f"- **{label} →** `{state}`"))

          if visualize:
              with graph_output:
                  visualize_automaton(automaton, title="DFA from JSON")
                  animate_path([s for _, s in steps], automaton)

          display(trace_output)
          display(graph_output)


        elif automaton_type == "NFA":
            # --- NFA Simulation ---
            result, paths = automaton.run(input_string)
            display(Markdown(f"### ✅ NFA Result: {'**Accepted ✅**' if result else '**Rejected ❌**'}"))
            display(Markdown("#### 🔎 NFA Accepting Path(s)"))
            if paths:
                for i, p in enumerate(paths):
                    display(Markdown(f"**Path {i+1}:** `{' → '.join(map(str, p))}`"))
            else:
                display(Markdown("_No valid paths found._"))

            if visualize:
                with graph_output:
                    visualize_automaton(automaton, title="NFA from JSON", is_nfa=True)
                    for p in paths:
                        animate_path(p, automaton, is_nfa=True)

            display(trace_output)
            display(graph_output)

            # --- DFA from NFA (only after NFA stuff is shown) ---
            if show_dfa:
                trace_output.clear_output()
                graph_output.clear_output()

                display(Markdown("---"))
                display(Markdown("### 🔁 Equivalent DFA (Subset Construction from NFA)"))

                dfa = nfa_to_dfa(automaton)
                dfa_result, dfa_steps = dfa.run_step_by_step(input_string)

                display(Markdown(f"#### 🎯 DFA Result: {'**Accepted ✅**' if dfa_result else '**Rejected ❌**'}"))
                display(Markdown("#### 🔍 DFA Step-by-Step Trace"))
                with trace_output:
                    for symbol, state in dfa_steps:
                        label = "Start" if symbol == '' else f"`{symbol}`"
                        display(Markdown(f"- **{label} →** `{state}`"))

                if visualize:
                    with graph_output:
                        visualize_automaton(dfa, title="Equivalent DFA (Converted from NFA)")
                        animate_path([s for _, s in dfa_steps], dfa)

                display(trace_output)
                display(graph_output)

    except ValueError as e:
        display(Markdown(f"**Error:** `{e}`"))
    except json.JSONDecodeError:
        display(Markdown("**Error:** Invalid JSON format. Please check your syntax.**"))

def render_ui():
    base = [automaton_type, input_string, visualize]
    if automaton_type.value == "NFA":
        base += [show_dfa]
    return VBox(base)


# Bind UI and simulation
ui = interactive_output(simulate, {
    "automaton_type": automaton_type,
    "input_string": input_string,
    "visualize": visualize,
    "show_dfa": show_dfa
})

# Re-render UI dynamically when type changes
def rerender_ui(change):
    clear_output(wait=True)
    display(render_ui())
    display(ui)

automaton_type.observe(rerender_ui, names='value')

# Initial render
display(render_ui())
display(ui)


Output()